# Classificador

Neste Workshop, vamos aprender como criar um classificador que diz quais músicas são da Rihanna e quais são da Beyoncé utilizando aprendizado de máquina, mais especificamente *Aprendizado Supervisionado*, uma das áreas de Machine Learning.

Então vamos lá!



## Importando os dados

O primeiro passo é importar seus dados, no caso nosso Dataframe (como chamamos a *tabela* que guarda as informações que usaremos). Fazemos isso com a biblioteca `pandas`.

Com a função `read_csv` lemos nosso arquivo e guardamos ele na váriavel `df`.



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('beyonce_rihanna.csv', index_col=0)

# vamos explorar nosso dataframe olhando apenas as primeiras linhas com a função abaixo:
df.head()

## Pré-processamentos

Antes de partir para o aprendizado de máquina, precisamos preparar nosso texto. Fazemos isso porque, para a máquina, algumas palavras ou estruturas do nosso texto não importam e não fazem diferença. 
São muitos os métodos de pré-processamento, mas aqui vamos realizar apenas alguns: 
* Tokenização
* Remover stopwords
* Deixar todo o texto em minúsculo
* Selecionar apenas letras com REGEX
* Lemmatização

In [ ]:
# Utilizando uma música como exemplo
exemplo = df['letra'][10]
exemplo

### Tokenização 

Uma parte importante no pré-processamento de um texto é a tokenização. Isto é, transformar elementos do seu texto em tokens, ou seja, strings dentro de uma lista  -  ou, se você não tiver conhecimento de python, transformar todas as palavras do texto em elementos individuais separados por aspas. 
Podemos tokenizar palavras com `word_tokenize`, essa função recebe o texto como argumento e retorna todas as palavras do texto em forma de tokens.



In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
#Tokenizando a primeira música
tokens = word_tokenize(exemplo)
tokens

### Selecionando apenas as letras e deixando todas em minúsculas

Para a máquina, pontuações não são necessárias, por isso um pré-processamento necessário é selecionar apenas as letras de um texto. 

Porém, **antes disso** precisamos deixar todas as letras em minúsculo, não somente porque isso facilita a aplicação do REGEX, mas também porque a máquina tende a interpretar palavras com letras maiúsculas e minúsculas como sendo diferentes. Por exemplo, Beyoncé e beyoncé podem ser interpretadas como palavras distintas. Então vamos deixar as letras minúsculas com a função `.lower`.

Feito isso, podemos selecionar apenas as letras com REGEX, mais especificamente com a função `re.findall`, que, além de retornar apenas as letras, já tokeniza o texto para você! 

 

In [ ]:
import re
letras = re.findall(r'\b[A-zÀ-úü]+\b', exemplo.lower())
letras

### Stopwords

Stopwords são palavras que, apesar de muito frequentes, não são importantes/relevantes para a máquina. Entre elas, podemos encontrar artigos como “o” e “uma”, ou preposições como “de” e “em”, entre outras palavras frequentes no idioma. Para removê-las do texto, utilizamos uma lista de stopwords disponível na biblioteca NLTK.

In [ ]:
from nltk.corpus import stopwords
stops = stopwords.words('english')
stops #lista de stopwords em inglês 

Como remover stopwords:

In [ ]:
sem_stopwords = [palavra for palavra in letras if palavra not in stops]
palavras_importantes = " ".join(sem_stopwords)
palavras_importantes

### Lematização 

Assim como Stopwords, ter verbos conjugados em um texto não faz diferença quando a máquina vai processá-lo. Por isso, existem duas ferramentas chamadas Lemmatização e Stemmatização. Ambas fazem a mesma coisa: Quando passado um texto como argumento, elas reduzem todas as formas verbais conjugadas à sua raiz. A única diferença, entretanto, é que a função que lemmatiza seu texto reduz todos os verbos a forma verdadeira da raiz  -  por isso quanto maior seu texto, mais tempo essa função demora para rodar no código - , enquanto a função que stemmatiza apenas "corta" as palavras no meio usando a raiz como base, o que pode gerar palavras que não existem.

In [ ]:
!python3 -m spacy download en_core_web_sm

In [ ]:
import spacy
spc = spacy.load('en_core_web_sm')

In [ ]:
spc_letras = spc(palavras_importantes)
lemmas = [token.lemma_ if token.pos_ == 'VERB' else str(token) for token in spc_letras]
texto_limpo = " ".join(lemmas)
print(texto_limpo)

Construa agora uma função para realizar todos os pré-processamentos ao invés de fazê-los um a um: 

In [ ]:
def limpar_texto(texto):
    '''
    Função para converter todas as letras para sua forma minúscula, selecionar apenas as letras,
    remover stopwords e lematizar o texto. 
    '''
    
    ### Transforme as letras para minúscula ###
    minusculas = ...
    
    ### Selecione apenas as letras do texto ##
    letras = ... 
    
    ### Removendo as stopwords ###
    stops = set(stopwords.words('english')) 
    # Retire as stopwords de letras
    palavras_sem_stopwords = ...
    # Junte as palavras sem stopwords 
    palavras_importantes = ... 
    
    ### Lematização ###
    spc_letras = spc(palavras_importantes)
    # Lematize o texto 
    lemmas = ...
    # Junte os lemmas 
    texto_limpo = ...
    
    return texto_limpo

Agora vamos aplicá-la aos nossos dados, mais espcificamente na coluna "letra", que contém as músicas:

In [ ]:
df['Texto Limpo'] = df['letra'].apply(limpar_texto)

In [ ]:
df.head() # vamos ver como ficou?

## Feature Extraction
Antes de treinar o nosso modelo, precisamos organizar os nossos documentos em features que o computador consegue entender, assim, vamos precisamos transformar o nosso texto em algum tipo de representação numérica. Para isso, vamos usar o Bag of Words. 

### Bag of Words 
**O que é o Bag of Words?:** BoW é uma forma de representação de texto que descreve a ocorrência de palavras em um documento. Para o BoW a ordem não importa, essa forma de representação só se importa se as palavras conhecidas ocorrem ou não no documento (literalmente um "saco" de palavras). 

Para implementarmos o Bag of Words, precisamos de três coisas: 
1. Um vocabulário com as palavras conhecidas
2. A ocorrência dessas palavras
3. Formar vetores a partir dos documentos 

**Exemplo**

"to the left to the left everything you own in the box to the left"

1. Construir o vocabulário

    ["to", "the", "left", "everything", "you", "own", "in", "box"]
    

2. Ocorrência das palavras

    {"to": 3, "the": 3, "left":3, "everything":1, "you":1, "own":1, "in":1, "box":1}


3. Vetores

    Considerando que o nosso documento fosse: "to the left"

    Usando o vocabulário que construímos antes, o nosso vetor seria: 

    [1, 1, 1, 0, 0, 0, 0]

### Count Vectorizer 
Felizmente, temos o CountVectorizer! Com ele, conseguimos implementar todos os passos acima de uma maneira bem simples: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

# Bag of words
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(df['Texto Limpo'])

Olhando o nosso vocabulário: 

In [ ]:
count_vectorizer.get_feature_names() #Todas as palavras do nosso vocabulário 

In [ ]:
count_vectorizer.vocabulary_.get('love')

Exemplo da nossa matriz termo-documento:

In [ ]:
df_cv = pd.DataFrame(X.toarray(), columns = count_vectorizer.get_feature_names())
df_cv.head()

No dataframe acima, cada uma das colunas representa uma das palavras do nosso vocabulário, e cada linha, um dos nossos documentos, ou seja, uma das nossas músicas. 


## Separando em Treino e Teste

In [ ]:
from sklearn.model_selection import train_test_split

X = X.toarray()
y = df['artista']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Naive Bayes


In [ ]:
from sklearn.naive_bayes import MultinomialNB

#Criando o Modelo Naive Bayes 
naive_bayes = MultinomialNB()

#.......Treinando o Modelo.......
naive_bayes.fit(X_train, y_train)

#Fazendo as previsões
naive_bayes_pred = naive_bayes.predict(X_test)

## Métricas 
Após estarmos com nosso modelo de classificação pronto, devemos avaliá-lo e, para isso, utilizamos as métricas de classificação. 
### Matriz de confusão
Primeiro, quando estamos lidando com um modelo cuja target é categórica (como no nosso caso, em que as músicas pertencem ou a Beyoncé ou a Rihanna), podemos utilizar uma matriz de confusão para analisarmos melhor onde o nosso modelo está acertando e onde ele está errando. Ela apresenta o seguinte formato:

<img src="https://www.researchgate.net/profile/Fabio_Araujo_Da_Silva/publication/323369673/figure/fig5/AS:597319787479040@1519423543307/Figura-13-Exemplo-de-uma-matriz-de-confusao.png" alt="Exemplo de uma matriz de confusão"/></a>

Na vertical, estão indicados os valores previstos pelo modelo e, na horizontal, os valores reais. Para cada elemento da matriz, temos dois valores associados: o previsto e o real. Se esse valores coincidirem, tem-se uma previsão correta/verdadeira (por exemplo, verdadeiros positivos e verdadeiros negativos, que estão em verde na imagem). Caso contrário, tem-se um erro cometido pelo modelo (como ocorre nos quadrados vermelhos da imagem acima). 

### Acurácia
A acurácia é, basicamente, uma métrica que indica a relação entre quanto o seu modelo acertou do quanto ele avaliou. Considerando a matriz de confusão mostrada, a acurácia seria igual à soma dos verdadeiros positivos com os verdadeiros negativos dividida pelo total (soma dos verdadeiros e falsos positivos e negativos). A acurácia não é uma boa métrica a ser utilizada quando analisamos dados desbalanceados, porque pode acontecer de o modelo prever muito bem o evento mais usual e ser péssimo prevendo o evento raro. Assim, como trata-se de uma média simples de acertos pelo total, a grande quantidade de acertos na previsão do evento mais usual compensaria a baixa taxa de acerto do evento raro, resultando em uma acurácia alta que não reflete corretamente a qualidade de predição do modelo.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

#Calculando a acurácia
acc = accuracy_score(naive_bayes_pred, y_test)

#Matriz de confusão 
cm = confusion_matrix(naive_bayes_pred, y_test)

print("Acurácia do modelo", acc)
print("\nMatriz de confusão: \n", cm)

## Avaliando as músicas
Agora, você pode tentar testar o seu modelo com alguma frase e ver a qual cantora ela se assemelha mais: 

In [ ]:
nova_frase = ["coloque sua frase aqui"] 
teste = count_vectorizer.transform(nova_frase)
pred = naive_bayes.predict(teste)
print(pred)